## Artifact Dependencies

This report includes graph visualization(s) using JavaScript and might not be exportable to some document formats.

### References

- [neovis.js (GitHub)](https://github.com/neo4j-contrib/neovis.js)
- [vis-network (GitHub)](https://github.com/visjs/vis-network)
- [vis network documentation](https://visjs.github.io/vis-network/docs/network)
- [Neo4j Graph Algorithms Jupyter Notebooks (GitHub)](https://github.com/neo4j-graph-analytics/graph-algorithms-notebooks)
- [Neo4j Graph Data Science Topological Sort](https://neo4j.com/docs/graph-data-science/current/algorithms/alpha/topological-sort)


In [1]:
import os
from neo4j import GraphDatabase
from IPython.core.display import Javascript, HTML
import json

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

neo4jUri = "bolt://localhost:7687"
neo4jUser = "neo4j"
neo4jPassword = os.environ.get("NEO4J_INITIAL_PASSWORD")

# Create the database driver to validate the connection
with GraphDatabase.driver(uri=neo4jUri, auth=(neo4jUser, neo4jPassword)) as driver:
    driver.verify_connectivity()

In [3]:
def neo4j_server_configuration(password, uri="bolt://localhost:7687", user="neo4j"):
    return {
        "neo4j": {
            "serverUrl": uri,
            "serverUser": user,
            "serverPassword": password
        }
    }

In [4]:
def visualization_configuration(node_distance: int = 200):
    return {
        "visConfig": {
            "nodes": {
                "shape": "hexagon",
                "shadow": False,
                "font": {
                    "strokeWidth": 4,
                    "strokeColor": "#F2F2FF",
                    "size": 12
                },
                "size": 22,
                "borderWidth": 2,
                "widthConstraint": {
                    "maximum": 60
                }
            },
            "edges": {
                "arrows": {
                    "to": { 
                        "enabled": True,
                        "scaleFactor": 0.3
                    }
                },
                "scaling": {
                    "max": 6
                }
            },
            "physics": {
                "hierarchicalRepulsion": {
                    "nodeDistance": node_distance, # 120
                    "centralGravity": 0.2, # 0.0
                    "springLength": 100, # 100
                    "springConstant": 0.02, # 0.01
                    "damping": 0.09, # 0.09
                    "avoidOverlap": 0.9 # 0
                },
                "solver": "hierarchicalRepulsion" # barnesHut
            },
            "layout": {
                "hierarchical": {
                    "enabled": True,
                    "sortMethod": "directed"
                }
            }
        }
    }

In [5]:
cssTemplate = """
.graph-visualization {
    width: 660px;
    height: 660px;
    border: 1px solid lightgray;
}
div.vis-tooltip {
  font-size: 6px;
}
"""

# Use JavaScript library neovis.js to render the graph into the HTML above
javascriptTemplate = """
function draw(NeoVis) {
  configuration.labels[NeoVis.NEOVIS_DEFAULT_CONFIG] = {
    [NeoVis.NEOVIS_ADVANCED_CONFIG]: {
      function: {
        title: NeoVis.objectToTitleHtml // Show all node properties in the tooltip
      }
    }
  }
  configuration.relationships[NeoVis.NEOVIS_DEFAULT_CONFIG] = {
    [NeoVis.NEOVIS_ADVANCED_CONFIG]: {
      function: {
        title: NeoVis.objectToTitleHtml // Show all relationship properties in the tooltip
      }
    }
  }
  configuration.labels.Artifact = {
      [NeoVis.NEOVIS_ADVANCED_CONFIG]: {
          function: {
              // Use "fileName" as label. Remove leading slash, trailing ".jar" and version number.
              // TODO Enrich the Graph so that there is a distinct property for the "cleaned up" artifact name
              label: (node) => node.properties.fileName.replace('/', '').replace('.jar', '').replace(/-[\d\\.]+/, '')
          }
      }
  }
  console.debug(configuration)
  const neoViz = new NeoVis.default(configuration);
  neoViz.render();
}

// Use JavaScript library neovis.js to render the graph into the HTML above
requirejs(['https://unpkg.com/neovis.js@2.1.0'], function(NeoVis){ 
  draw(NeoVis);
}, function (err) {
    throw new Error("Failed to load NeoVis:" + err);
});
"""

htmlTemplate="""
<!DOCTYPE html>
<html>
<head>
  <title>Jupyter Notebook embedded neovis.js visualization</title>
  <style type="text/css">{css}</style>
</head>
<body>
  <div id="{containerId}" class="graph-visualization"></div>
  <script type="text/javascript" defer>
    {script}
  </script>
</body>
</html>
"""

## Dependencies Hierarchy

The following hierarchical graphs shows dependencies with the most used and shared elements at the bottom and the ones that use the most dependencies on top. The visualization is limited to the first 20 nodes and their direct dependency ordered descending by their layer ("maxDistanceFromSource"). 

For the whole list of topologically sorted elements including the hierarchical layer see the detailed report `TopologicalSorted....csv`. It is for example helpful to find out in which order Artifacts need to be build/assembled in case of breaking changes.

### Hierarchical Java Artifact Dependencies

The following Graph shows up to 60 Java Artifact dependencies in hierarchical form sorted by their topology.

In [6]:
def java_artifact_query_configuration():
    query = """
        MATCH (artifact:Java:Artifact:Archive)-[dependency:DEPENDS_ON]->(dependent:Java:Artifact:Archive)
        WHERE  artifact.maxDistanceFromSource IS NOT NULL
        AND  dependent.maxDistanceFromSource > artifact.maxDistanceFromSource
        RETURN artifact, dependency, dependent
        ORDER BY artifact.maxDistanceFromSource DESC
                ,artifact.maxDistanceFromSource ASC
                ,artifact.topologicalSortIndex  ASC
                ,dependent.topologicalSortIndex ASC
        LIMIT 20        
    """
    
    return {
        "initialCypher": query,
        "labels": {
            "Artifact": {
                "label": "fileName"
            },
        },
        "relationships": {
            "DEPENDS_ON": {
                "value": "weight",
                "label": False
            }
        }
    }

In [7]:
htmlElement = {"containerId": "graph-visualization-java-artifacts"}
serverConfiguration = neo4j_server_configuration(uri=neo4jUri, user=neo4jUser,password=neo4jPassword)

# Assemble the neovis.js configuration by joining the different parts of it
graphVisualizationConfiguration = {**htmlElement, **visualization_configuration(), **serverConfiguration, **java_artifact_query_configuration()}

# Assemble the HTML by including CSS and Javascript
jsonConfiguration = json.dumps(graphVisualizationConfiguration)
javascriptContent="configuration=" + jsonConfiguration +"; " + javascriptTemplate
htmlContent = htmlTemplate.format(script=javascriptContent, css=cssTemplate, containerId=htmlElement["containerId"])

# Display the HTML
display(HTML(htmlContent))

In [8]:
import time
time.sleep(6)